In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from math import sqrt
import os

# ============================================================
# 1) CARGA DE DATOS
# ============================================================
csv_path = 'landsat_features_validation.csv'
df = pd.read_csv(
    csv_path,
    engine='python',
    na_values=['', ' ', 'NA', 'NaN', 'nan'],
    skip_blank_lines=True
)

# Normalizar nombres y eliminar columnas totalmente vacías
df.columns = [c.strip() for c in df.columns]
for col in list(df.columns):
    if df[col].isna().all():
        df.drop(columns=[col], inplace=True)

# Parseo de fecha
if 'Sample Date' in df.columns:
    df['Sample Date'] = pd.to_datetime(df['Sample Date'], dayfirst=True, errors='coerce')

# Coerción a numérico
num_cols = ['Latitude','Longitude','nir','green','swir16','swir22','NDMI','MNDWI']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Crear ID de sitio
df['site_id'] = df['Latitude'].round(5).astype(str) + ',' + df['Longitude'].round(5).astype(str)

# ============================================================
# 2) CALCULAR ÍNDICES NDMI Y MNDWI
# ============================================================
df['NDMI_calc'] = (df['nir'] - df['swir16']) / (df['nir'] + df['swir16'])
df['MNDWI_calc'] = (df['green'] - df['swir16']) / (df['green'] + df['swir16'])

# ============================================================
# 3) VALIDACIÓN DE CONSISTENCIA ENTRE ÍNDICES
# ============================================================
metrics = {}
for provided, calc in [('NDMI','NDMI_calc'), ('MNDWI','MNDWI_calc')]:
    sub = df[[provided, calc]].dropna()
    diff = sub[provided] - sub[calc]
    metrics[provided] = {
        'MAE': diff.abs().mean(),
        'RMSE': sqrt((diff**2).mean()),
        'Corr': sub.corr().iloc[0,1],
        'n': len(sub)
    }
metrics_df = pd.DataFrame(metrics).T

# ============================================================
# 4) ESTADÍSTICOS Y MATRICES
# ============================================================
missing = df.isna().mean().sort_values(ascending=False).to_frame('proporcion_nulos')
numeric_cols_present = ['nir','green','swir16','swir22','NDMI','MNDWI','NDMI_calc','MNDWI_calc']
describe_df = df[numeric_cols_present].describe().T
corr_df = df[numeric_cols_present].corr()

# ============================================================
# 5) DETECCIÓN DE OUTLIERS (IQR)
# ============================================================
outlier_flags = pd.DataFrame(index=df.index)
for c in numeric_cols_present:
    series = df[c].dropna()
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    outlier_flags[c] = (df[c] < low) | (df[c] > high)

df['es_outlier'] = outlier_flags.any(axis=1)
outliers_df = df[df['es_outlier']]

# ============================================================
# 6) EXPORTAR TABLAS A DISCO
# ============================================================
out_dir = 'salidas_analisis'
os.makedirs(out_dir, exist_ok=True)

missing.to_csv(os.path.join(out_dir, '01_proporcion_nulos.csv'))
describe_df.to_csv(os.path.join(out_dir, '02_resumen_estadistico.csv'))
corr_df.to_csv(os.path.join(out_dir, '03_matriz_correlaciones.csv'))
metrics_df.to_csv(os.path.join(out_dir, '04_consistencia_indices.csv'))
outliers_df.to_csv(os.path.join(out_dir, '05_registros_atipicos.csv'), index=False)

# ============================================================
# 7) GENERACIÓN DE GRÁFICAS
# ============================================================
plt.style.use('seaborn-v0_8-whitegrid')

# ----------------- Histogramas -----------------
fig, axes = plt.subplots(
    nrows=int(np.ceil(len(numeric_cols_present)/3)),
    ncols=3,
    figsize=(15, 4 * int(np.ceil(len(numeric_cols_present)/3)))
)
axes = axes.flatten()

for i, col in enumerate(numeric_cols_present):
    axes[i].hist(df[col].dropna(), bins=30, color="#4e79a7")
    axes[i].set_title(f"Distribución de {col}")

for j in range(i+1, len(axes)):
    axes[j].axis("off")

fig.tight_layout()
fig.savefig(os.path.join(out_dir, "G1_histogramas.png"), dpi=150)
plt.close(fig)

# ----------------- Boxplots -----------------
# ----------------- Boxplots -----------------
fig, ax = plt.subplots(figsize=(12, 6))
ax.boxplot(
    [df[c].dropna() for c in numeric_cols_present],
    tick_labels=numeric_cols_present,   # <--- CORREGIDO
    patch_artist=True,
    boxprops=dict(facecolor="#59a14f")
)
ax.set_title("Boxplots bandas e índices")
fig.tight_layout()
fig.savefig(os.path.join(out_dir, "G2_boxplots.png"), dpi=150)
plt.close(fig)

# ----------------- Heatmap correlación -----------------
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.imshow(corr_df.values, cmap="coolwarm", vmin=-1, vmax=1)
ax.set_xticks(range(len(corr_df.columns)))
ax.set_xticklabels(corr_df.columns, rotation=45)
ax.set_yticks(range(len(corr_df.index)))
ax.set_yticklabels(corr_df.index)
fig.colorbar(cax)
fig.tight_layout()
fig.savefig(os.path.join(out_dir, "G3_correlacion.png"), dpi=150)
plt.close(fig)

# ----------------- NDMI vs MNDWI -----------------
fig, ax = plt.subplots(figsize=(7, 6))
sub = df[['NDMI','MNDWI']].dropna()
ax.scatter(sub['NDMI'], sub['MNDWI'], s=18, color="#e15759", alpha=0.6)
ax.axhline(0, ls="--", color="gray")
ax.axvline(0, ls="--", color="gray")
ax.set_title("Relación NDMI vs MNDWI")
fig.tight_layout()
fig.savefig(os.path.join(out_dir, "G4_dispersion_NDMI_MNDWI.png"), dpi=150)
plt.close(fig)

# ----------------- Series temporales -----------------
top_sites = df.dropna(subset=['Sample Date']).groupby('site_id').size().sort_values(ascending=False).head(4).index

for metric in ['NDMI', 'MNDWI']:
    fig, axes = plt.subplots(nrows=len(top_sites), ncols=1, figsize=(10, 3.2 * len(top_sites)), sharex=True)
    if len(top_sites) == 1:
        axes = [axes]

    for ax, sid in zip(axes, top_sites):
        sub = df[df['site_id']==sid].dropna(subset=['Sample Date', metric]).sort_values('Sample Date')
        ax.plot(sub['Sample Date'], sub[metric], marker='o')
        ax.set_title(f"{metric} - sitio {sid}")

    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"G5_series_{metric}.png"), dpi=150)
    plt.close(fig)

# ----------------- Mapa NDMI medio -----------------
site_mean = df.groupby('site_id').agg({'Latitude':'first','Longitude':'first','NDMI':'mean'}).dropna()

fig, ax = plt.subplots(figsize=(7,6))
sc = ax.scatter(site_mean['Longitude'], site_mean['Latitude'], c=site_mean['NDMI'], cmap='viridis', s=60)
fig.colorbar(sc).set_label("NDMI medio")
ax.set_title("NDMI medio por sitio")
fig.tight_layout()
fig.savefig(os.path.join(out_dir, "G6_mapa_NDMI_medio.png"), dpi=150)
plt.close(fig)

print("✓ Análisis completado. Revisa la carpeta 'salidas_analisis/'.")

✓ Análisis completado. Revisa la carpeta 'salidas_analisis/'.


In [5]:
# ============================================================
# MAPA DE CALIDAD DE AGUA BASADO EN MNDWI (Índice de Agua)
# ============================================================

import pandas as pd
import folium

# --- 1. Cargar dataset --------------------------------------

df = pd.read_csv("landsat_features_validation.csv")
df["Sample Date"] = pd.to_datetime(df["Sample Date"], dayfirst=True, errors="coerce")

# Asegurar columnas numéricas
df["MNDWI"] = pd.to_numeric(df["MNDWI"], errors="coerce")
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

df = df.dropna(subset=["Latitude", "Longitude", "MNDWI"])

# --- 2. Función de colores según niveles de MNDWI -------------

def color_por_mndwi(value):
    """
    Clasificación típica de MNDWI para evaluar agua superficial:
    >0.1       = Alta probabilidad de agua (azul)
    0 a 0.1    = Húmedo / transicional (verde)
    -0.2 a 0   = Seco–moderado (amarillo)
    < -0.2     = Muy seco / sin agua (rojo)
    """
    if value > 0.1:
        return "blue"
    elif value > 0:
        return "green"
    elif value > -0.2:
        return "yellow"
    else:
        return "red"

# --- 3. Crear mapa centrado en el dataset --------------------

lat_center = df["Latitude"].mean()
lon_center = df["Longitude"].mean()

m = folium.Map(location=[lat_center, lon_center], zoom_start=6)

# --- 4. Agregar puntos coloreados por MNDWI ------------------

for _, row in df.iterrows():
    valor = row["MNDWI"]
    color = color_por_mndwi(valor)

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=(
            f"<b>MNDWI:</b> {valor:.3f}<br>"
            f"<b>Fecha:</b> {row['Sample Date'].date()}"
        )
    ).add_to(m)

# --- 5. Agregar leyenda --------------------------------------

legend_html = """
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 220px; height: 160px;
    background-color: white; z-index:9999;
    font-size:14px; border:2px solid grey; border-radius:8px; padding:10px;
">
<b>Niveles de MNDWI</b><br>
<i style="background:blue; width:12px; height:12px; display:inline-block;"></i> Agua (>0.1)<br>
<i style="background:green; width:12px; height:12px; display:inline-block;"></i> Húmedo (0 a 0.1)<br>
<i style="background:yellow; width:12px; height:12px; display:inline-block;"></i> Seco (-0.2 a 0)<br>
<i style="background:red; width:12px; height:12px; display:inline-block;"></i> Muy seco (< -0.2)<br>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

# --- 6. Guardar mapa -----------------------------------------

m.save("mapa_mndwi_landsat.html")
print("Mapa guardado como 'mapa_mndwi_landsat.html'")

Mapa guardado como 'mapa_mndwi_landsat.html'
